In [17]:
import numpy as np
import tensorly as tl
from scipy.io import savemat
from itertools import product
from tensorly.decomposition import tucker
from sklearn.model_selection import KFold
from tensorly.tenalg import mode_dot, multi_mode_dot, kronecker
import torch

tl.set_backend("pytorch")
torch.set_default_tensor_type(torch.DoubleTensor)
torch.set_default_dtype(torch.float64)

In [18]:
# 生成数据
from generate_data import generate_complex
# 找最佳 lam 和 R
from pipeline import do_testing, compute_q2_hopls_coef
# 找lam、R 下的系数tensor
from hopls import HOPLS

# 生成数据

In [19]:
np.random.seed(42)
snr = 0  # 这个是噪声
I1 = 20 # 这个是样本量？ 比如这里 X的维数应该是 （20, 20, 10）
In = [20, 10] # X 的维数
Jm = [10, 5] # Y的维数 必须>=2
n_folds = 5 # 交叉验证的折数
X, Y = generate_complex(I1, In, Jm, R=5, snr=snr) # 生成数据

In [20]:
X = np.array(X)
Y = np.array(Y) 

# 找出最好的lambda

In [21]:
re = do_testing(n_folds, X, Y, lambda_max=10, R_max=20)
re['HOPLS_L'] # 五折CV中，每次最好的lambda

/Users/huahuacui/opt/anaconda3/envs/SDA/lib/python3.9/site-packages/tensorly/backend/core.py:1106: UserWarning: In partial_svd: converting to NumPy. Check SVD_FUNS for available alternatives if you want to avoid this.
  warnings.warn('In partial_svd: converting to NumPy.'
/Users/huahuacui/opt/anaconda3/envs/SDA/lib/python3.9/site-packages/tensorly/backend/core.py:1106: UserWarning: In partial_svd: converting to NumPy. Check SVD_FUNS for available alternatives if you want to avoid this.
  warnings.warn('In partial_svd: converting to NumPy.'
/Users/huahuacui/opt/anaconda3/envs/SDA/lib/python3.9/site-packages/tensorly/backend/core.py:1106: UserWarning: In partial_svd: converting to NumPy. Check SVD_FUNS for available alternatives if you want to avoid this.
  warnings.warn('In partial_svd: converting to NumPy.'
/Users/huahuacui/opt/anaconda3/envs/SDA/lib/python3.9/site-packages/tensorly/backend/core.py:1106: UserWarning: In partial_svd: converting to NumPy. Check SVD_FUNS for available alt

[6, 7, 7, 7, 7]

# 得到系数矩阵

In [24]:
# 把数据分开，输出每一折的系数 inter 和 y_pred
cv = KFold(n_folds)
iterm = 0
y_pred_re_set = {} # 存储 每一折的y的预测值 跟Y的维数不一样！
coef_re_set = {} # 存储每一折对应的系数
for train_idx, valid_idx in cv.split(X, Y):
    X_train = torch.Tensor(X[train_idx])
    Y_train = torch.Tensor(Y[train_idx])
    X_valid = torch.Tensor(X[valid_idx])
    Y_valid = torch.Tensor(Y[valid_idx])
    
    y_pred, coef = compute_q2_hopls_coef(X_train, Y_train, X_valid, Y_valid, re['HOPLS_L'][iterm], R_max=20)
    y_pred_re_set[iterm] = y_pred
    coef_re_set[iterm] = coef
    iterm += 1

In [25]:
y_pred_re_set[0]

tensor([[[-12.7713, -11.4978,  -2.5941,  -0.3787,  10.2208],
         [  1.9195, -21.9814,  -9.6142,   5.6537,  -3.5261],
         [-14.9187, -12.1742,   4.5367,  -7.8741,  21.4622],
         [ -0.4975,  18.9122,  -7.1031,   0.4299,  -1.1878],
         [  6.9412,  14.3053,  -4.3814,   1.4280,   0.8009],
         [ 18.5290,  -2.2760,  -6.6161,   2.5227,  -3.6905],
         [  7.4753, -13.3187,   3.5357,   3.2689,  -7.2165],
         [-20.5417, -13.9914,  11.1619,  -6.5084,   9.4386],
         [  3.3286, -18.3493,  -0.3761,   2.7841,  -3.2159],
         [ -0.9633,  40.8084,  -5.4698,  -1.8226,   2.8900]],

        [[  0.4856,   4.0418,  -1.1763,  -1.9973,   4.5246],
         [ -1.1470,   0.6842,   1.5959,  -1.5925,   1.9418],
         [ -0.9663,  -2.3360,   2.5244,   1.0360,  -5.3427],
         [  1.0769,  -1.5664,  -0.6346,   1.9225,  -3.8398],
         [  0.3967,  -1.1884,   3.3044,   0.4109,  -1.5667],
         [ -3.2507,  -6.3719,   6.9424,   0.1156,  -3.4117],
         [ -0.8060,   

In [26]:
coef_re_set[0]

tensor([[-2.8911e-03, -3.1588e-04, -1.4147e-03,  ..., -4.6630e-04,
          1.8488e-03, -4.3484e-03],
        [ 1.2649e-02, -7.4503e-03,  1.3486e-02,  ...,  6.5451e-03,
         -7.5777e-03,  7.0539e-03],
        [ 8.1391e-03,  1.2469e-03,  1.2738e-03,  ...,  5.5757e-03,
         -4.1866e-03,  1.2610e-02],
        ...,
        [ 1.9478e-03, -1.1659e-03,  5.2741e-04,  ...,  1.2886e-02,
         -2.2182e-02,  1.2318e-02],
        [ 6.6261e-05,  2.7791e-03, -2.8802e-03,  ..., -1.3262e-04,
          4.5706e-04,  2.7294e-03],
        [ 4.7876e-03, -4.3163e-03,  6.3756e-03,  ...,  4.3772e-03,
         -6.7109e-03,  4.0016e-03]])